# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [1]:
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00


In [2]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm
import torch

In [3]:
! nvidia-smi

Mon Dec 30 07:32:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
torch.random.manual_seed(0)
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
def generate_content(prompt):
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
  )

  generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.0,
      "do_sample": False,
  }
  output = pipe(prompt, **generation_args)
  print(output[0]['generated_text'])

We select those papers that have been featured in Hugging Face papers.

In [6]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

Code to extract text from PDFs.

In [7]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

Summarizing the papers.

In [12]:
prompt = "Summarize this research article into a table highlighting its strengths and weaknesses in two different columns. "
for paper in tqdm(papers):
    try:
        paper["summary"] = generate_content(prompt + extract_pdf(paper["url"])).text
    except Exception as e:
        print("Generation failed", e)
        paper["summary"] = "Paper not available"
    torch.cuda.empty_cache()


  0%|          | 0/6 [00:00<?, ?it/s]Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 17%|█▋        | 1/6 [00:01<00:07,  1.51s

Generation failed CUDA out of memory. Tried to allocate 23.36 GiB. GPU 0 has a total capacity of 14.75 GiB of which 8.17 GiB is free. Process 14348 has 6.58 GiB memory in use. Of the allocated memory 5.85 GiB is allocated by PyTorch, and 624.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Device set to use cuda
 33%|███▎      | 2/6 [00:02<00:04,  1.25s/it]

Generation failed CUDA out of memory. Tried to allocate 18.05 GiB. GPU 0 has a total capacity of 14.75 GiB of which 8.81 GiB is free. Process 14348 has 5.93 GiB memory in use. Of the allocated memory 5.28 GiB is allocated by PyTorch, and 541.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Device set to use cuda
 50%|█████     | 3/6 [00:05<00:05,  1.93s/it]

...

Generation failed CUDA out of memory. Tried to allocate 7.87 GiB. GPU 0 has a total capacity of 14.75 GiB of which 2.48 GiB is free. Process 14348 has 12.26 GiB memory in use. Of the allocated memory 11.92 GiB is allocated by PyTorch, and 221.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Device set to use cuda
 83%|████████▎ | 5/6 [00:07<00:01,  1.54s/it]

Generation failed CUDA out of memory. Tried to allocate 8.40 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.84 GiB is free. Process 14348 has 12.90 GiB memory in use. Of the allocated memory 12.51 GiB is allocated by PyTorch, and 277.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Device set to use cuda
100%|██████████| 6/6 [00:09<00:00,  1.52s/it]

Generation failed CUDA out of memory. Tried to allocate 17.99 GiB. GPU 0 has a total capacity of 14.75 GiB of which 8.82 GiB is free. Process 14348 has 5.93 GiB memory in use. Of the allocated memory 5.28 GiB is allocated by PyTorch, and 538.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


We print the results to a html file.

In [13]:
def convert_markdown_to_html_table(markdown_table):
    lines = markdown_table.strip().split("\n")
    headers = ["Strengths", "Weaknesses"]  # Fix the header line
    rows = lines[3:]

    html_table = "<table border='1'>\n<thead>\n<tr>"
    html_table += "".join(f"<th>{header.strip()}</th>" for header in headers)
    html_table += "</tr>\n</thead>\n<tbody>\n"

    for row in rows:
        if set(row.strip()) == {'|', '-'}:
            continue
        cells = row.split("|")[1:-1]
        html_table += "<tr>" + "".join(f"<td>{cell.strip().replace('**', '<b>').replace('**', '</b>')}</td>" for cell in cells) + "</tr>\n"

    html_table += "</tbody>\n</table>"
    return html_table

In [14]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {model_name}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    html_table = convert_markdown_to_html_table(paper["summary"])
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{html_table}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)

We can also print the results to this notebook as markdown.

In [15]:
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[HuatuoGPT-o1, Towards Medical Complex Reasoning with LLMs](https://arxiv.org/pdf/2412.18925)**<br>Paper not available<br><br>

**[Orient Anything: Learning Robust Object Orientation Estimation from Rendering 3D Models](https://arxiv.org/pdf/2412.18605)**<br>Paper not available<br><br>

**[Task Preference Optimization: Improving Multimodal Large Language Models with Vision Task Alignment](https://arxiv.org/pdf/2412.19326)**<br>Paper not available<br><br>

**[SBS Figures: Pre-training Figure QA from Stage-by-Stage Synthesized Images](https://arxiv.org/pdf/2412.17606)**<br>Paper not available<br><br>

**[From Elements to Design: A Layered Approach for Automatic Graphic Design Composition](https://arxiv.org/pdf/2412.19712)**<br>Paper not available<br><br>

**[VideoMaker: Zero-shot Customized Video Generation with the Inherent Force of Video Diffusion Models](https://arxiv.org/pdf/2412.19645)**<br>Paper not available<br><br>